# Poverty and inequality effects of new unemployment benefits, payroll tax cuts, and universal payments 

*By [Max Ghenis](https://www.ubicenter.org/about#h.p_T-QXperbv_x2) and
[Nate Golden](https://www.ubicenter.org/about#h.p_T-QXperbv_x2)*

*July 2020*

## Abstract

The expiration of the \$600-per-week Federal Pandemic Unemployment Compensation (FPUC) program
at the end of July 2020 has spurred discussions for new Covid-19 relief legislation in the United States.
Proposals for this bill include extending FPUC throughout 2020,
cutting payroll taxes, and unconditional payments.
We use data from the Current Population Survey between 2009 and 2018
to estimate the poverty and inequality effects
of a hypothetical FPUC and FPUC extension,
compared to budget-neutral payroll tax cuts and universal payments,
respectively.
Our simulations show that a universal payment would have lowered
poverty and inequality similarly to April-to-July FPUC
(i.e., the policy that's expiring).
However, universal payments would have reduced poverty and inequality
significantly more than August-to-December FPUC extensions;
for example, it would have reduced poverty twice as much in 2009.
Payroll tax cuts would have had much smaller effects than either policy,
and universal payments that include children are consistently more
effective than adult-only payments.


## Background

The CARES Act established two major new unemployment insurance reforms:
Pandemic Unemployment Assistance (PUA), which expands unemployment benefits to more worker categories,
and Federal Pandemic Unemployment Compensation (FPUC), which adds a federally-funded
\$600 per week to all unemployment benefits.
While PUA is available for up to 39 weeks through the end of the year,
FPUC runs only from [April through July](https://wdr.doleta.gov/directives/attach/UIPL/UIPL_15-20.pdf).

Given FPUC's expiration and the Congressional Budget Office's [projections](In July 2020, the Congressional Budget Office [projected](https://www.cbo.gov/data/budget-economic-data#4)
that the unemployment rate would remain above 10 percent throughout 2020.),
federal lawmakers are preparing for a new relief package.
Congressional Democrats are seeking to extend FPUC in full through January, as specified in
the House-passed [HEROES Act](https://www.washingtonpost.com/us-policy/2020/05/12/house-democrats-coronavirus-3-trillion/).
The Trump administration has proposed [cutting payroll taxes](https://www.washingtonpost.com/us-policy/2020/07/16/payroll-tax-cut-trump-coronavirus/),
and members of both parties have spoken of another round of direct payments,
following the Recovery Rebates in the CARES Act.

FPUC's work disincentives have made it controversial: it replaces
[over 100 percent of wages](https://colab.research.google.com/drive/11hRHMXiEYmuvEXKZZoHK9qjePFMk5Nzy) for
all workers earning below \\$21 per hour, and depending on the state, for workers earning up to \\$33 per hour.[^replacement]
Capping the wage replacement rate was infeasible due to limitations with state unemployment benefit
computer systems.
Its proponents emphasize income stabilization and the need to reduce work to prevent Covid infections.
To the extent that FPUC may be making it harder to find workers, payroll tax cuts can offset that effect.

[^replacement]: Assumes that workers have worked full-time at a constant wage for the past several quarters.

Distributional effects have been less prominent in the debates over Covid relief policies;
this paper aims to remedy that.


## Data

We use the [Current Population Survey March Supplement](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.html)
from 2009 to 2018, extracted from the [IPUMS](https://cps.ipums.org/) service.
The [Supplemental Poverty Measure](https://www.census.gov/topics/income-poverty/supplemental-poverty-measure.html) was introduced in 2009,
while 2018 is the latest survey year available as of this writing.

We extract fields used for tax and poverty analysis, as well as the "[number of weeks](https://cps.ipums.org/cps-action/variables/WKSUNEM1#description_section),
in single weeks, that the respondent looked for work or was on layoff during the preceding calendar year."
This field allows us to estimate the total FPUC benefits for each respondent.

[TODO:] All dollar values are scaled to 2020 dollars using the CPI-U-RS price deflator.

## Methodology



### Imputing FPUC unemployment benefits

The CPS only provides the number of weeks each respondent was unemployed, not the specific calendar weeks.
Since the specific time periods--
April to July to model past policy, and August to December to model proposed policy--
are relevant for the simulation, we impute the calendar weeks of unemployment for each respondent.
We assume that the unemployment spell is continuous, and therefore only affected but the
starting week, which we assign randomly as week $0$ to week $52 - WeeksUnemployed$, to ensure that the
unemployment spell is fully within the survey year.

We then calculate the total weeks from April to July and from August to December when the person
was unemployed, and multiply those numbers by \$600 to simulate FPUC or the FPUC extension, respectively.


### Tax simulation

We use the open-source [Tax-Calculator](https://pslmodels.github.io/Tax-Calculator/)
software for simulating FPUC tax liabilities.
This involves grouping people into tax units[^taxunits] and calculating tax-relevant
fields from each tax unit's aggregate CPS responses.
We then calculate each tax unit's federal tax liability with and without FPUC--
applying 2020 tax law for all years--and store the difference as the FPUC tax.

[^taxunits]: Our code for tax unit creation was based on similar code by
[Ernie Tedeschi](https://github.com/ernietedeschi/tcpoverty),
[Sam Portnow](https://users.nber.org/~taxsim/to-taxsim/cps/cps-portnow/TaxSimRScriptForDan.R),
and the Policy Simulation Library's
[Tax-Data project](https://github.com/PSLmodels/taxdata/tree/master/cps_data),
which creates tax units from non-IPUMS CPS records.

To identify the net-of-tax FPUC benefits for each person,[^taxunitperson]
we allocate  the tax unit's FPUC tax across individuals in proportion to their FPUC benefits.
This leaves us with the net-of-tax FPUC benefits for each person in the sample.

[^taxunitperson]: We need the tax liability for each person because
some tax units include people from multiple SPM units.

### Poverty estimation

SPM poverty rates are calculated by grouping individuals into *SPM units*,
which are sub-household groups that the Bureau of Labor Statistics identifies
as sharing resources.
Each SPM unit then has its own poverty threshold---based on composition and
housing situation---and resources---
based on income, taxes, mandatory spending categories, and transfers.
SPM units are determined to classified as poor if their poverty threshold
exceeds their resources.

To calculate this, we group each person into their SPM unit.



In [21]:
"""
SETUP
"""

import numpy as np
import pandas as pd
import taxcalc as tc
import microdf as mdf
import plotly.express as px
import pandas_datareader as pdr  # For CPI series.

# For setting a random period of unemployment given a person's duration.
np.random.seed(0)

person = pd.read_csv('ui/data/cps.csv.gz')

"""
# Prepare data

* Loads and preprocesses IPUMS ASEC from 2009 to 2018
* Simulates FPUC unemployment benefits
* Calculates tax liability from unemployment benefits via `taxcalc`
* Calculates budget-neutral UBI and payroll tax cuts
* Aggregates to SPM unit level
* Joins back to person record
* Exports person and SPM unit records
"""

"""
## PREPROCESSING
"""

%run 'ui/convert_asec_taxcalc.py'
%run 'ui/make_tax_units.py'

# Set columns to lowercase and to 0 or null as appropriate.
prep_ipum(person)
# Add taxid and related fields.
tax_unit_id(person)
# Add other person-level columns in taxcalc form.
person = convert_asec_person_taxcalc(person)
# 99 is the missing code for wksunem1.
# Note: Missing codes for features used in taxcalc are recoded in
# convert_asec_taxcalc.py.
person.loc[person.wksunem1 == 99, 'wksunem1'] = 0
# The 2014 file was released in two ways, so weights must be halved.
person.asecwt *= np.where(person.year == 2014, 0.5, 1)
person.spmwt *= np.where(person.year == 2014, 0.5, 1)

# Adjust all dollar values to 2020.
cpiu = pdr.get_data_fred('CPIAUCSL', '2009-01-01')
# Filter to January.
cpiu = cpiu[cpiu.index.month == 1]
cpiu.index = cpiu.index.year
# Multiplier to 2020.
cpiu['inflate2020'] = cpiu.CPIAUCSL[2020] / cpiu.CPIAUCSL
# Survey year is the reported year minus 1.
person['FLPDYR'] = person.year - 1
person = person.merge(cpiu[['inflate2020']], left_on='FLPDYR',
                      right_index=True)
DOLLAR_VALS = ['spmtotres', 'spmthresh',
               'e02400', 'e01700', 'e00300', 'e02300', 'e00650', 'incrent',
               'p23250', 'fica', '_e00200', 'e00600', 'e01500', 
               'e00200p', 'e00200s', 'e00200']
person[DOLLAR_VALS] = person[DOLLAR_VALS] * person.inflate2020

"""
## Add UI to person records

Assume that unemployment blocks are contiguous and randomly distributed.
"""
person['ui_start'] = np.random.randint(1, 53 - person.wksunem1,
                                       person.shape[0])
person['ui_end'] = person.ui_start + person.wksunem1

FPUC_START = 13  # April was the 13th week.
FPUC_MAX_WEEKS = 17  # April to July.
FPUC2_START = FPUC_START + FPUC_MAX_WEEKS
FPUC2_MAX_WEEKS = 22  # August to December.
FPUC_WEEKLY_BEN = 600
person['fpuc_weeks'] = np.fmax(
    0, np.fmin(person.ui_end - FPUC_START,
               np.fmin(person.wksunem1, FPUC_MAX_WEEKS)))
person['fpuc2_weeks'] = np.fmax(
    0, np.fmin(person.ui_end - FPUC2_START,
               np.fmin(person.wksunem1, FPUC2_MAX_WEEKS)))
person['fpuc'] = FPUC_WEEKLY_BEN * person.fpuc_weeks
person['fpuc2'] = person.fpuc + FPUC_WEEKLY_BEN * person.fpuc2_weeks

# Checks
assert person.fpuc_weeks.max() == FPUC_MAX_WEEKS
assert person.fpuc2_weeks.max() == FPUC2_MAX_WEEKS
assert person.fpuc_weeks.min() == person.fpuc2_weeks.min() == 0

# Store original unemployment benefits.
person['e02300_orig'] = person.e02300

"""
## Create tax units and calculate tax liability
"""
person['RECID'] = person.FLPDYR * 1e9 + person.taxid

def get_taxes(tu):
    """ Calculates taxes by running taxcalc on a tax unit DataFrame.
    
    Args:
        tu: Tax unit DataFrame.
    
    Returns:
        Series with tax liability for each tax unit.
    """
    return mdf.calc_df(records=tc.Records(tu, weights=None, gfactors=None),
                       # year doesn't matter without weights or gfactors.
                       year=2020).tax.values

# Create tax unit dataframe.
tu = create_tax_unit(person)
tu['tax'] = get_taxes(tu)

# Simulate FPUC.

# Create tax unit dataframe.
person.e02300 = person.e02300_orig + person.fpuc
tu_fpuc = create_tax_unit(person)
tu['e02300_fpuc'] = tu_fpuc.e02300
tu['tax_fpuc'] = get_taxes(tu_fpuc)
del tu_fpuc

# Simulate extended FPUC.

# Create tax unit dataframe.
person.e02300 = person.e02300_orig + person.fpuc2
tu_fpuc2 = create_tax_unit(person)
tu['e02300_fpuc2'] = tu_fpuc2.e02300
tu['tax_fpuc2'] = get_taxes(tu_fpuc2)
del tu_fpuc2

# Change person e02300 back.
person.e02300 = person.e02300_orig

"""
## Merge back to the person level

Have each person pay the share of tax differences in proportion with their
FPUC.
"""

tu['fpuc_total'] = tu.e02300_fpuc - tu.e02300
tu['fpuc2_total'] = tu.e02300_fpuc2 - tu.e02300
tu['fpuc_tax_total'] = tu.tax_fpuc - tu.tax
tu['fpuc2_tax_total'] = tu.tax_fpuc2 - tu.tax

person = person.merge(tu[['RECID', 'fpuc_total', 'fpuc2_total',
                          'fpuc_tax_total', 'fpuc2_tax_total']],
                      on='RECID')

for i in ['fpuc', 'fpuc2']:
    person[i + '_tax'] = np.where(person[i + '_total'] == 0, 0,
        person[i + '_tax_total'] * person[i] / person[i + '_total'])
    person[i + '_net'] = person[i] - person[i + '_tax']
    
# Checks that the totals match by person and tax unit, then garbage-collect.
assert np.allclose(tu.fpuc_total.sum(), person.fpuc.sum())
assert np.allclose(tu.fpuc2_total.sum(), person.fpuc2.sum())
assert np.allclose(tu.fpuc_tax_total.sum(), person.fpuc_tax.sum())
assert np.allclose(tu.fpuc2_tax_total.sum(), person.fpuc2_tax.sum())
del tu

"""
## Calculate budget-neutral UBIs and payroll taxes
"""

def single_year_summary(year):
    fpuc_budget = mdf.weighted_sum(person[person.FLPDYR == year],
                                   'fpuc_net', 'asecwt')
    fpuc1_2_budget = mdf.weighted_sum(person[person.FLPDYR == year],
                                      'fpuc2_net', 'asecwt')
    fpuc2_budget = fpuc1_2_budget - fpuc_budget
    pop = person[person.FLPDYR == year].asecwt.sum()
    adult_pop = person[(person.FLPDYR == year) &
                       (person.age > 17)].asecwt.sum()
    total_fica = mdf.weighted_sum(person[person.FLPDYR == year],
                                  'fica', 'asecwt')
    fpuc_ubi = fpuc_budget / pop
    fpuc_adult_ubi = fpuc_budget / adult_pop
    fpuc_fica_pct_cut = 100 * fpuc_budget / total_fica
    # Note: FPUC2 includes FPUC1.
    fpuc2_ubi = fpuc2_budget / pop
    fpuc2_adult_ubi = fpuc2_budget / adult_pop
    fpuc2_fica_pct_cut = 100 * fpuc2_budget / total_fica
    return pd.Series([fpuc_budget, fpuc2_budget, pop, adult_pop, total_fica,
                      fpuc_ubi, fpuc_adult_ubi, fpuc_fica_pct_cut,
                      fpuc2_ubi, fpuc2_adult_ubi, fpuc2_fica_pct_cut])

OVERALL_YEARLY_METRICS = ['fpuc_budget', 'fpuc2_budget', 'pop', 'adult_pop',
                          'total_fica']
FPUC_YEARLY_METRICS = ['fpuc_ubi', 'fpuc_adult_ubi', 'fpuc_fica_pct_cut']
FPUC2_YEARLY_METRICS = ['fpuc2_ubi', 'fpuc2_adult_ubi', 'fpuc2_fica_pct_cut']
all_metrics = (
    OVERALL_YEARLY_METRICS + FPUC_YEARLY_METRICS + FPUC2_YEARLY_METRICS)
DISPLAY_METRICS = {
    'fpuc_budget': 'Cost of FPUC',
    'fpuc2_budget': 'Cost of expanding FPUC',
    'pop': 'Population',
    'adult_pop': 'Adult population',
    'total_fica': 'Total FICA',
    'fpuc_ubi': 'Universal one-time payment (FPUC)',
    'fpuc_adult_ubi': 'Adult one-time payment (FPUC)',
    'fpuc_fica_pct_cut': 'FICA % cut (FPUC)',
    'fpuc2_ubi': 'Universal one-time payment (FPUC2)',
    'fpuc2_adult_ubi': 'Adult one-time payment (FPUC2)',
    'fpuc2_fica_pct_cut': 'FICA % cut (FPUC2)'
}

year_summary = pd.DataFrame({'FLPDYR': person.FLPDYR.unique()})
year_summary[all_metrics] = year_summary.FLPDYR.apply(single_year_summary)

person = person.merge(
    year_summary[['FLPDYR'] + FPUC_YEARLY_METRICS + FPUC2_YEARLY_METRICS],
    on='FLPDYR')

"""
Run calculations on all fields (except `fpuc_ubi` which already works).
"""

# Zero out adult UBIs for children.
person.loc[person.age < 18, 'fpuc_adult_ubi'] = 0
# Calculate total FICA cut by multiplying FICA by % cut.
# Divide by 100 as it was previously multiplied by 100 for table displaying.
person['fpuc_fica_cut'] = person.fica * person.fpuc_fica_pct_cut / 100
# Similar process for FPUC2, but also adding fpuc_net since this is on top
# of the existing FPUC.
person['fpuc2_ubi'] = person.fpuc_net + person.fpuc2_ubi
person['fpuc2_adult_ubi'] = (person.fpuc_net + 
                             np.where(person.age > 17,
                                      person.fpuc2_adult_ubi, 0))
person['fpuc2_fica_cut'] = (person.fpuc_net +
                             person.fica * person.fpuc2_fica_pct_cut / 100)

"""
Verify the `fpuc` and `fpuc2` have equal costs, respectively, in each year.
"""
for year in person.FLPDYR.unique():
    tmp = person[person.FLPDYR == year]
    fpuc = mdf.weighted_sum(tmp, 'fpuc_net', 'asecwt')
    assert np.allclose(fpuc, mdf.weighted_sum(tmp, 'fpuc_ubi', 'asecwt'))
    assert np.allclose(fpuc, 
                       mdf.weighted_sum(tmp, 'fpuc_adult_ubi', 'asecwt'))
    assert np.allclose(fpuc, mdf.weighted_sum(tmp, 'fpuc_fica_cut', 'asecwt'))
    fpuc2 = mdf.weighted_sum(tmp, 'fpuc2_net', 'asecwt')
    assert np.allclose(fpuc2, mdf.weighted_sum(tmp, 'fpuc2_ubi', 'asecwt'))
    assert np.allclose(fpuc2, 
                       mdf.weighted_sum(tmp, 'fpuc2_adult_ubi', 'asecwt'))
    assert np.allclose(fpuc2, mdf.weighted_sum(tmp,
                                               'fpuc2_fica_cut', 'asecwt'))
del tmp

"""
## Aggregate to SPM units
"""

SPM_COLS = ['FLPDYR', 'spmfamunit', 'spmtotres', 'spmthresh', 'spmwt']
CHG_COLS = ['fpuc_net', 'fpuc_ubi', 'fpuc_adult_ubi', 'fpuc_fica_cut',
            'fpuc2_net', 'fpuc2_ubi', 'fpuc2_adult_ubi', 'fpuc2_fica_cut']
spmu = person.groupby(SPM_COLS)[CHG_COLS].sum().reset_index()
for i in CHG_COLS:
    spmu['spmtotres_' + i] = spmu.spmtotres + spmu[i]
    
"""
## Map back to persons
"""
# Shrink the data.
person = person[['asecwt', 'age', 'race', 'sex'] + CHG_COLS + SPM_COLS]

spm_resource_cols = ['spmtotres_' + i for i in CHG_COLS]
SPMU_MERGE_COLS = ['spmfamunit', 'FLPDYR']
person = person.merge(spmu[SPMU_MERGE_COLS + spm_resource_cols],
                      on=SPMU_MERGE_COLS)
# Poverty flags.
for i in CHG_COLS:
    person['spmpoor_' + i ] = person['spmtotres_' + i] < person.spmthresh
# Also calculate baseline.
person['spmpoor'] = person.spmtotres < person.spmthresh

SPM_OUTCOLS = SPM_COLS + spm_resource_cols
spmu = spmu[SPM_OUTCOLS]

PERSON_OUTCOLS = (['asecwt', 'age', 'race', 'sex', 'spmpoor'] + 
                  CHG_COLS + spm_resource_cols + SPM_COLS +
                  ['spmpoor_' + i for i in CHG_COLS])
person = person[PERSON_OUTCOLS]


# Print overall summary
print("All figures in millions.")
(year_summary.set_index('FLPDYR')[OVERALL_YEARLY_METRICS].rename(
    columns=DISPLAY_METRICS) / 1e6).round(1)

MemoryError: Unable to allocate 27.6 TiB for an array with shape (1948983, 1948983) and data type float64

In [ ]:
# Print reform parameter summary.
(year_summary.set_index('FLPDYR')[
    FPUC_YEARLY_METRICS + FPUC2_YEARLY_METRICS].rename(
    columns=DISPLAY_METRICS)).round(2)

## Results

### FPUC thus far

In most of the last decade, FPUC would have reduced poverty more than the alternatives.
However, in 2009 and 2010, when economic conditions most closely resembled today's,
universal transfers were more effective.

In all years, universal transfers and FPUC outperformed payments to adults only,
and especially outperformed the payroll tax cut.

In [ ]:
def pov(reform, year, age_group='All', race='All'):
    """ Calculate the poverty rate under the specified reform for the
        specified population.
        Note: All arguments refer to the poverty population, not the reform.
    
    Args:
        reform: One of CHG_COLS. If None, provides the baseline rate.
        year: Year of the data (year before CPS survey year).
        age_group: Age group, either
            - 'Children' (under 18)
            - 'Adults' (18 or over)
            - 'All'
        race: Race code to filter to. Defaults to 'All'.
        
    Returns:
        2018 SPM poverty rate.
    """
    # Select the relevant poverty column for the reform.
    if reform == 'baseline':
        poverty_col = 'spmpoor'
    else:
        poverty_col = 'spmpoor_' + reform
    # Filter by year.
    target_persons = person[person.FLPDYR == year]
    # Filter by age group.
    if age_group == 'Children':
        target_persons = target_persons[target_persons.age < 18]
    elif age_group == 'Adults':
        target_persons = target_persons[target_persons.age >= 18]
    # Filter by race.
    if race != 'All':
        target_persons = target_persons[target_persons.race == race]
    # Return poverty rate (weighted average of poverty flag).
    return mdf.weighted_mean(target_persons, poverty_col, 'asecwt')


def pov_row(row):
    """ Calculate poverty based on parameters specified in the row.
    
    Args:
        row: pandas Series.
        
    Returns:
        2018 SPM poverty rate.
    """
    return pov(row.reform, row.year, row.age_group, row.race)

pov_rates = mdf.cartesian_product({'reform': ['baseline'] + CHG_COLS,
                                   'year': person.FLPDYR.unique(),
                                   'age_group': ['All', 'Children', 'Adults'],
                                   'race': ['All', 200]})  # 200 means Black.
pov_rates['pov'] = 100 * pov_rates.apply(pov_row, axis=1)

"""
### Poverty gap and inequality
Calculate these for all people and SPM units, without breaking out by age or
race.
"""

def pov_gap_b(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    tmp = spmu[spmu.FLPDYR == year]
    pov_gap = np.maximum(tmp.spmthresh - tmp[resource_col], 0)
    return (pov_gap * tmp.spmwt).sum() / 1e9

def pov_gap_row(row):
    return pov_gap_b(row.reform, row.year)

pov_gap_ineq = pov_rates[['reform', 'year']].drop_duplicates()
pov_gap_ineq['pov_gap_b'] = pov_gap_ineq.apply(pov_gap_row, axis=1)

"""
### Inequality

By individual based on their percentage of SPM resources.
"""

def gini(reform, year):
    if reform == 'baseline':
        resource_col = 'spmtotres'
    else:
        resource_col = 'spmtotres_' + reform
    tmp = person[person.FLPDYR == year]
    return mdf.gini(tmp[resource_col], tmp.asecwt)

def gini_row(row):
    return gini(row.reform, row.year)

pov_gap_ineq['gini'] = pov_gap_ineq.apply(gini_row, axis=1)

"""
## Postprocess

Create columns for displaying and grouping each reform.
"""

REFORM_DISPLAY = {
    'baseline': 'Baseline',
    'fpuc_net': '$600 per week UI',
    'fpuc_ubi': 'Payment to everyone',
    'fpuc_adult_ubi': 'Payment to adults',
    'fpuc_fica_cut': 'Payroll tax cut',
    'fpuc2_net': 'Extend $600 per week',
    'fpuc2_ubi': 'Payment to everyone',
    'fpuc2_adult_ubi': 'Payment to adults',
    'fpuc2_fica_cut': 'Payroll tax cut'
}

REFORM_GROUP = {
    'baseline': 'Baseline',
    'fpuc_net': 'fpuc',
    'fpuc_ubi': 'fpuc',
    'fpuc_adult_ubi': 'fpuc',
    'fpuc_fica_cut': 'fpuc',
    'fpuc2_net': 'fpuc2',
    'fpuc2_ubi': 'fpuc2',
    'fpuc2_adult_ubi': 'fpuc2',
    'fpuc2_fica_cut': 'fpuc2'
}

for i in [pov_rates, pov_gap_ineq]:
    i['reform_display'] = i.reform.map(REFORM_DISPLAY)
    i['reform_group'] = i.reform.map(REFORM_GROUP)
    i['baseline'] = np.where(i.reform_group == 'fpuc', 'baseline', 'fpuc_net')
    
"""
### Calculate % changes from relevant baselines
"""

POV_RATES_KEYS = ['year', 'age_group', 'race']  # Plus reform/baseline
POV_GAP_INEQ_KEYS = ['year']
BASELINES = ['baseline', 'fpuc_net']

pov_rates_baselines = pov_rates[pov_rates.reform.isin(
    BASELINES)][POV_RATES_KEYS + ['reform', 'pov']]
pov_rates_baselines.rename(columns={'reform': 'baseline',
                                    'pov': 'baseline_pov'}, inplace=True)
pov_rates2 = pov_rates[pov_rates.reform != 'baseline'].merge(
    pov_rates_baselines, on=POV_RATES_KEYS + ['baseline'])
pov_rates2['pov_pc'] = 100 * (pov_rates2.pov / pov_rates2.baseline_pov - 1)

pov_gap_ineq_baselines = pov_gap_ineq[
    pov_gap_ineq.reform.isin(BASELINES)][
    POV_GAP_INEQ_KEYS + ['reform', 'pov_gap_b', 'gini']]
pov_gap_ineq_baselines.rename(columns={'reform': 'baseline',
                                       'pov_gap_b': 'baseline_pov_gap_b',
                                       'gini': 'baseline_gini'}, inplace=True)
pov_gap_ineq2 = pov_gap_ineq[pov_gap_ineq.reform != 'baseline'].merge(
    pov_gap_ineq_baselines, on=POV_GAP_INEQ_KEYS + ['baseline'])
pov_gap_ineq2['pov_gap_pc'] = 100 * (pov_gap_ineq2.pov_gap_b /
                                     pov_gap_ineq2.baseline_pov_gap_b - 1)
pov_gap_ineq2['gini_pc'] = 100 * (pov_gap_ineq2.gini /
                                  pov_gap_ineq2.baseline_gini - 1)

## Charts

# Colors from https://material.io/design/color/the-color-system.html
BLUE = '#1976D2'
GRAY = '#BDBDBD'
RED = '#C62828'
LIGHT_BLUE = '#64B5F6'

COLOR_MAP = {
    '$600 per week UI': GRAY,
    'Extend $600 per week': GRAY,
    'Payroll tax cut': RED,
    'Payment to everyone': BLUE,
    'Payment to adults': LIGHT_BLUE
}

def line_graph(df, group, y, yaxis_title, title,
               x='year', color='reform_display', xaxis_title=''):
    """Style for line graphs.
    
    Arguments
        df: DataFrame with data to be plotted.
        group: Reform group, either 'fpuc' (against baseline), or
            'fpuc2' (against FPUC baseline).
        y: Column name to plot on the y axis.
        yaxis_title: y axis title.
        title: Plot title.
        x: The column name for the x axis. Defaults to 'year'.
        color: The string representing the column to show different colors of.
            Defaults to 'reform_display'.
        xaxis_title: x axis title. Defaults to '' (since the year is obvious).
    
    Returns
        Nothing. Shows the plot.
    """
    df = df[df.reform_group == group]
    if group == 'fpuc':
        yaxis_title += ' from baseline'
        title += ' from baseline'
    else:
        yaxis_title += ' from Apr-Jul FPUC baseline'
        title += ' moving forward'
    is_pc = y[-3:] == '_pc'
    if is_pc:
        df = df.round(2)
    fig = px.line(df, x=x, y=y, color=color, color_discrete_map=COLOR_MAP)
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        font=dict(family='Roboto'),
        hovermode='x',
        plot_bgcolor='white',
        legend_title_text='' 
    )
    if y == 'pov_gap_b':
        fig.update_layout(yaxis_tickprefix='$', yaxis_ticksuffix='B')
    elif y == 'pov_rate' or is_pc:  # Rate or percent changes.
        fig.update_layout(yaxis_ticksuffix='%')
    if is_pc:
        # Calculate a range to show so that the 0% zeroline is visible.
        ymin = df[y].min()
        ymax = df[y].max()
        if ymax < 0:
            ymax = 0
        yrange = ymax - ymin
        ymin_vis = ymin - 0.1 * yrange
        ymax_vis = ymax + 0.1 * yrange
        fig.update_yaxes(zeroline=True, zerolinewidth=0.5, 
                         zerolinecolor='lightgray',
                         range=[ymin_vis, ymax_vis])

    fig.update_traces(mode='markers+lines', hovertemplate=None)

    fig.show()

# Round for the hover text.
line_graph(pov_rates2[(pov_rates2.age_group == 'All') &
                      (pov_rates2.race == 'All')],
           group='fpuc', y='pov_pc',
           yaxis_title='Change in SPM poverty rate',
           title='Poverty reduction by reform')

The poverty gap is an alternative poverty measure which quantifies the total
amount we'd have to distribute to end poverty.
Since we would have to give more to people who are farther below the poverty line,
this measure goes beyond the poverty rate in also measuring the severity of poverty.

<note: add a trend of the total poverty gap?>

Over the past decade, each of FPUC, universal transfers, and adult-only transfers reduce 
the poverty gap similarly--and far more than the payroll tax cut.
As in the poverty rate case, the universal transfers slightly outperform in the
2009 recession, while FPUC slightly outperforms in the rest of the period. 

In [ ]:
# Round for the hover text.
line_graph(pov_gap_ineq2, group='fpuc', y='pov_gap_pc',
           yaxis_title='Change in poverty gap',
           title='Poverty gap by reform')

Moving beyond poverty, we found that FPUC would have reduced overall inequality
(as measured by the Gini index over individuals, considering their SPM unit's total resources)
more than the universal payments, by a small but consistent margin.
If anything, the payroll tax cut would slightly increase inequality.

In [ ]:
line_graph(pov_gap_ineq2, group='fpuc', y='gini_pc',
           yaxis_title='Change in Gini index',
           title='Inequality by reform')

### Moving forward

While FPUC and universal transfers performed similarly in the baseline case,
universal transfers reduce poverty and inequality significantly more than expanding
FPUC, given that FPUC had already been in effect from April through July.
The difference is larger in worse economic times (2009 vs. 2018),
when using the poverty gap measure to consider poverty depth, and is especially
pronounced for child and Black poverty.

The intuition behind these trends is that FPUC focuses resources on unemployed people,
especially long-term unemployed. While these can be similarly poor to people not in the labor force,
who constitute about half of the population in poverty, the first FPUC round helped this targeted
group substantially. To reduce overall poverty, future resources are better spread out
than adding more eggs to the unemployed basket.

Starting with the poverty rate, we find that extending FPUC would have reduced
poverty by 3.9 percent in 2009, compared to a baseline where FPUC had already
been enacted from April through July. However, a budget-neutral universal payment
would have reduce poverty twice as much--by 7.8 percent.
An adult-only transfer would be somewhere in the middle, and a payroll tax cut
would reduce poverty by less than half as much as the FPUC extension.

In [ ]:
# Round for the hover text.
line_graph(pov_rates2[(pov_rates2.age_group == 'All') &
                      (pov_rates2.race == 'All')],
           group='fpuc2', y='pov_pc',
           yaxis_title='Change in SPM poverty rate',
           title='Poverty reduction by reform')

Considering poverty depth with the poverty gap measure shows that a universal payment
would have nearly three times the effect of extending FPUC.

In [ ]:
# Round for the hover text.
line_graph(pov_gap_ineq2, group='fpuc2', y='pov_gap_pc',
           yaxis_title='Change in poverty gap',
           title='Poverty gap by reform')

While the initial FPUC reduced inequality more than universal transfers in every year,
the opposite is true of the FPUC expansion.

In [ ]:
line_graph(pov_gap_ineq2, group='fpuc2', y='gini_pc',
           yaxis_title='Change in Gini index',
           title='Inequality by reform')

For both child and Black poverty, the universal payments have more than double
the effect of FPUC extension. Unsurprisingly, the adult-only restriction
dramatically reduces the effectiveness of the payments for reducing child poverty
specifically.

In [ ]:
line_graph(pov_rates2[(pov_rates2.age_group == 'Children') &
                      (pov_rates2.race == 'All')],
           group='fpuc2', y='pov_pc',
           yaxis_title='Change in child SPM poverty rate',
           title='Child poverty reduction by reform')

In [ ]:
line_graph(pov_rates2[(pov_rates2.age_group == 'All') &
                      (pov_rates2.race == 200)],
           group='fpuc2', y='pov_pc',
           yaxis_title='Change in Black SPM poverty rate',
           title='Black poverty reduction by reform')

## Conclusion

